## Importing BigQuery data into Neo4j

Importing from a relational database can be straightforward; table names become node labels, foreign keys become relationships, and many-to-many relationships become a relationship with many properties.

However, it is important to define the problem you are trying to solve to optimise the data model.  

In Neo4j, nouns that describe _things_ are node labels, the verbs become relationship types.  For example, an order may contain many products.

You can find modeling tips and tricks in the [Modeling Fundamentals course on GraphAcademy](https://graphacademy.neo4j.com/courses/importing-fundamentals/).


### Prerequisites

To import from BigQuery, you will need to set up authentication with Google Cloud with appropriate billing and permissions.  This notebook assumes that this has been set up, and the default application is defined for the machine.

Prerequisites:

* [gloud cli installed](https://cloud.google.com/sdk/docs/install)
    * macos/brew - `brew install gcloud-cli`
* `gcloud auth application-default login`

### Dependencies

You will need to install the following Pyhton libraries:

* `neo4j`
* `google-cloud-bigquery`
* `pandas`

You will also need `ipykernel` installed to use this jupyter notebook.


### Connect to Neo4j

To connect to Neo4j, use the `GraphDatase.driver()` method using the database URI, and authenticate with your username and password.

In [1]:
from neo4j import GraphDatabase

uri = 'neo4j://localhost:7687'
user = 'neo4j'
password = 'neo4jpassword'

driver = GraphDatabase.driver(
    uri,
    auth=(user, password)
)

# Verify the connection details are correct.  If not, an error will be thrown.
driver.verify_connectivity()

### Run a Cypher statement

Use the `driver.execute_query()` method, which expects a positional parameter for the Cypher statement.  Any named parameters not suffixed with an underscore are treated as parameters and can be accessed in the query using a `$`.


In [2]:
records, summary, keys = driver.execute_query(
    "MATCH (n) RETURN count(*) AS count, $foo AS parameter",
    foo="bar"
)

for record in records:
    for key in keys:
        print(f"{key}: {record[key]}")

count: 2145
parameter: bar


### Transform results into a Pandas dataframe

The `result_transformer_` parameter can be used to specify a function that modifies the output.  The `Result.to_df` method turns the into a pandas DataFrame.



In [3]:
from neo4j import Result

driver.execute_query("""
UNWIND range(1, 10) as id
RETURN id, id * 2 as double, randomUuid() AS uuid, rand() AS random
""", result_transformer_=Result.to_df)

,id,double,uuid,random
0,1,2,f00e158c-c540-4962-bcd1-2a559f115427,0.766435
1,2,4,dcb1111c-076a-4e8e-bd3d-0ff5b7fb58d1,0.454053
2,3,6,5e40c2d1-e23d-48c2-ab0b-cb22d569f03d,0.301551
3,4,8,68265bb4-9446-4f4e-be35-8222186c208e,0.616246
4,5,10,912a5755-61bc-4678-a076-af5614f8a061,0.561261
5,6,12,5ce9d7d9-04bd-47f6-a78a-724f2c5aec36,0.736301
6,7,14,43dc8653-ad79-435d-9f00-197a029071da,0.107102
7,8,16,7cca2ca3-03aa-49a7-a091-7ae3fe94c7dc,0.515184
8,9,18,c2245fd9-ce7a-4105-ac5d-48209d333a35,0.151806
9,10,20,ac98e81c-be06-407c-b996-48d9ee8adbca,0.018430


## Connect to BigQuery

This example assumes that default application credentials have been set for the machine.  For example:

```
gcloud auth application-default login
```

Use the query method to execute an SQL statement.  This query can return normalised, or denormalised data.


In [ ]:
from google.cloud import bigquery

bq_client = bigquery.Client(project="project-id")

In [7]:
# execute an SQL query
res = bq_client.query("SELECT * FROM `bigquery-public-data.new_york_citibike.citibike_trips` WHERE tripduration > 100 LIMIT 1000")
rows = [ dict(row) for row in res ]

In [8]:
# convert the data into a pandas DataFrame.
import pandas as pd

df = pd.DataFrame(rows)

df.head()

,tripduration,starttime,stoptime,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender,customer_plan
0,57174,2016-03-08 14:00:11,2016-03-09 05:53:06,248,Laight St & Hudson St,40.721854,-74.007718,3240,NYCBS Depot BAL - DYR,0.000000,0.000000,15750,Customer,NaN,unknown,
1,213,2015-12-18 07:05:32,2015-12-18 07:09:05,531,Forsyth St & Broome St,40.718939,-73.992663,3019,NYCBS Depot - DEL,40.716633,-73.981933,22175,Subscriber,1980.0,male,
2,14923,2016-01-28 19:02:59,2016-01-28 23:11:43,152,Warren St & Church St,40.714740,-74.009106,3019,NYCBS Depot - DEL,40.716633,-73.981933,17692,Subscriber,1986.0,male,
3,229405,2015-10-20 06:43:23,2015-10-22 22:26:49,529,W 42 St & 8 Ave,40.757570,-73.990985,3019,NYCBS Depot - DEL,40.716633,-73.981933,19731,Subscriber,1975.0,male,
4,1402974,2016-01-13 09:41:14,2016-01-29 15:24:09,354,Emerson Pl & Myrtle Ave,40.693631,-73.962236,3019,NYCBS Depot - DEL,40.716633,-73.981933,14665,Subscriber,1968.0,female,


## Import the list of dicts into Neo4j

Each row contains the following nodes: 

* Station (id, name, latitude, longitude)
* Bike (id)
* Ride (starttime, stoptime, duration)

And the following relationships:

* (:Ride)-[:STARTS_AT]->(:Station)
* (:Ride)-[:ENDS_AT]->(:Station)
* (:Ride)-[:USES]->(:Bike)

![Data Model](../docs/bike-ride-station-model.png)

Each of these should be imported into the database sequentially, first by creating the Nodes, then creating the relationships.  Each node will have a unique identifier, so it's a good idea to first create a unique constraint to avoid duplicate nodes.

### Create Unique Constraints

In [9]:
with driver.session() as session:
    session.run("CREATE CONSTRAINT IF NOT EXISTS FOR (s:Station) REQUIRE s.id IS UNIQUE")
    session.run("CREATE CONSTRAINT IF NOT EXISTS FOR (b:Bike) REQUIRE b.id IS UNIQUE")
    session.run("CREATE CONSTRAINT IF NOT EXISTS FOR (r:Ride) REQUIRE r.id IS UNIQUE")


## Import the data

When you import data into Neo4j, it is sensible to turn the result set into smaller batches, this avoids OOM errors.









In [10]:
batch_size = 100

def batch(cypher, iterable, batch_size=100):
    """
    Util function to incrementally import batches of data into Neo4j.
    This avoids out of memory errors due to larger sizes exceding the Java heap size.
    """
    while len(iterable) > 0:
        current_batch = iterable[:batch_size]
        driver.execute_query(cypher, rows=current_batch)
        iterable = iterable[batch_size:]


### Importing nodes

Start by importing nodes.


In [11]:
# Import the stations
stations = [
    dict(id=id, name=name, latitude=lat, longitude=lon)
    for id, name, lat, lon in {
        (row[k+'_station_id'], row[k+'_station_name'], row[k+'_station_latitude'], row[k+'_station_longitude'])
        for row in rows for k in ('start', 'end')
    }
]

batch(r"""
UNWIND $rows as row
MERGE (s:Station {id: row.id})
ON CREATE SET
    s.name = row.name,
    s.location = point({latitude: row.latitude, longitude: row.longitude})
""", stations, batch_size=100)


# Import the bikes
bikes = [
    dict(id=id)
    for id in set(row['bikeid'] for row in rows)
]

batch(r"""
UNWIND $rows as row
MERGE (b:Bike {id: row.id})
""", bikes, batch_size=100)

### Importing relationships

Then use the unique identifier columns to find the nodes, and create the relationships.


In [12]:
# (:Ride)-[:USES]->(:Bike)
# (:Ride)-[:STARTS_AT]->(:Station)
# (:Ride)-[:ENDS_AT]->(:Station)
batch(r"""
UNWIND $rows as row
MERGE (r:Ride {id: row['bikeid'] + toString(row['starttime']) + toString(row['stoptime'])})
SET r.tripduration = row.tripduration,
    r.starttime = localdatetime(row.starttime),
    r.stoptime = localdatetime(row.stoptime),
    r.usertype = row.usertype,
    r.birth_year = row.birth_year,
    r.gender = row.gender,
    r.customer_plan = row.customer_plan

MERGE (s:Station {id: row.start_station_id})
MERGE (e:Station {id: row.end_station_id})
MERGE (b:Bike {id: row.bikeid})
MERGE (r)-[:STARTS_AT]->(s)
MERGE (r)-[:ENDS_AT]->(e)
MERGE (r)-[:USES]->(b)
""", rows, batch_size=100)

## Querying the data

In [13]:
driver.execute_query("""
MATCH (s:Station)
RETURN s.name AS name, s.location AS location LIMIT 10
""", result_transformer_=Result.to_df)

,name,location
0,E 81 St & 3 Ave,"(-73.9567526, 40.77573034)"
1,E 71 St & 1 Ave,"(-73.95622730255127, 40.76703432309872)"
2,21 St & 43 Ave,"(-73.94594845, 40.75052534)"
3,W 51 St & 6 Ave,"(-73.98042047, 40.7606597)"
4,W 42 St & Dyer Ave,"(-73.99379968643188, 40.75898481399634)"
5,South St & Gouverneur Ln,"(-74.00670227, 40.70355377)"
6,St Marks Pl & 2 Ave,"(-73.98713956, 40.7284186)"
7,Cathedral Pkwy & Broadway,"(-73.96699104, 40.804213)"
8,E 82 St & 2 Ave,"(-73.95324922, 40.77517793)"
9,Broadway & W 55 St,"(-73.98192338, 40.7652654)"


In [14]:
# Most used bikes
driver.execute_query(r"""
MATCH (b:Bike)<-[:USES]-(r:Ride)
RETURN b.id AS id,
    count(*) AS rides,
    sum(r.tripduration) AS duration,
    round(1.0 * sum(r.tripduration) / count(*), 2) AS average_duration
ORDER BY duration DESC LIMIT 10
""", result_transformer_=Result.to_df)

,id,rides,duration,average_duration
0,14665,2,1404264,702132.0
1,19731,1,229405,229405.0
2,19587,1,179830,179830.0
3,15750,1,57174,57174.0
4,28972,1,33170,33170.0
5,17342,1,15021,15021.0
6,17692,1,14923,14923.0
7,22536,1,13359,13359.0
8,24303,1,11184,11184.0
9,20967,1,10532,10532.0


## Visualising the data

The data can be quickly visualised within a notebook using [yFiles Jupyter notebook plugin](https://github.com/yWorks/yfiles-jupyter-graphs-for-neo4j).

In [15]:
from yfiles_jupyter_graphs_for_neo4j import Neo4jGraphWidget

g = Neo4jGraphWidget(driver)

g.show_cypher("MATCH (s)-[r]->(t) RETURN s,r,t LIMIT 20")


GraphWidget(layout=Layout(height='720px', width='100%'))

Available `layout` parameter options for the `show_cypher` method: 

* circular
* hierarchic
* organic
* interactive_organic
* orthogonal
* radial
* tree
* map
* orthogonal_edge_router
* organic_edge_route